In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

train_path = '/content/drive/MyDrive/train.csv'
test_path = '/content/drive/MyDrive/test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print("Training Data Info:")
print(train_df.info())
print("\nTraining Data Description:")
print(train_df.describe())

print("\nTesting Data Info:")
print(test_df.info())

print("\nFirst 5 rows of Training Data:")
print(train_df.head())
print("\nFirst 5 rows of Testing Data:")
print(test_df.head())
#sum of samples that are not Nan/null in training data
print(train_df.notna().sum())
#sum of samples that are  Nan/null in test data
print(test_df.isna().sum())
#sum of samples that are  Nan/null in training data
print(train_df.isna().sum())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8148 entries, 0 to 8147
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        8148 non-null   int64  
 1   sequence  8148 non-null   object 
 2   target    8148 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 191.1+ KB
None

Training Data Description:
                 id       target
count   8148.000000  8148.000000
mean    5033.456554    52.302745
std     2924.877222     4.738780
min        0.000000    38.520141
25%     2500.750000    48.608375
50%     5020.500000    51.620779
75%     7555.750000    55.496006
max    10091.000000    69.875198

Testing Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1945 entries, 0 to 1944
Data columns (total 2 columns):
 #   Column    Non-Null Count 

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


vocab = "ARNDCQEGHILKMFPSTWYVU"

def one_hot_encode_seq(seq, vocab=vocab, max_length=None):
    """ One-hot encode a protein sequence, padding to max_length. """
    if max_length is None:
        max_length = len(seq)
    encoding = np.zeros((max_length, len(vocab)), dtype=int)
    aa_to_index = {aa: idx for idx, aa in enumerate(vocab)}
    for i, aa in enumerate(seq[:max_length]):  # Slice sequence if longer than max_length
        if aa in aa_to_index:
            encoding[i, aa_to_index[aa]] = 1
    return encoding.flatten()

def frequency_encode_seq(seq, vocab=vocab):
    """ Calculate frequency of each amino acid in a sequence. """
    encoding = np.zeros(len(vocab), dtype=float)
    aa_to_index = {aa: idx for idx, aa in enumerate(vocab)}
    seq_length = len(seq)
    for aa in seq:
        if aa in aa_to_index:
            encoding[aa_to_index[aa]] += 1
    if seq_length > 0:
        encoding /= seq_length
    return encoding

def encode_dataset(df, encoding_func, max_length=None):
    """ Apply encoding function to the 'sequence' column of a dataframe. """
    if 'max_length' in encoding_func.__code__.co_varnames:
        encoded_features = np.array([encoding_func(seq, vocab, max_length) for seq in df['sequence']])
    else:
        encoded_features = np.array([encoding_func(seq, vocab) for seq in df['sequence']])
    return encoded_features

max_length = max(train_df['sequence'].str.len().max(), test_df['sequence'].str.len().max())

train_features_one_hot = encode_dataset(train_df, one_hot_encode_seq, max_length)
test_features_one_hot = encode_dataset(test_df, one_hot_encode_seq, max_length)

train_features_freq = encode_dataset(train_df, frequency_encode_seq)
test_features_freq = encode_dataset(test_df, frequency_encode_seq)

train_targets = train_df['target'].values

In [8]:
model_one_hot = LinearRegression()
model_one_hot.fit(train_features_one_hot, train_targets)
train_predictions_one_hot = model_one_hot.predict(train_features_one_hot)

test_predictions_one_hot = model_one_hot.predict(test_features_one_hot)

model_freq = LinearRegression()
model_freq.fit(train_features_freq, train_targets)
train_predictions_freq = model_freq.predict(train_features_freq)

rmse_one_hot = np.sqrt(mean_squared_error(train_targets, train_predictions_one_hot))

rmse_freq = np.sqrt(mean_squared_error(train_targets, train_predictions_freq))

print("RMSE with One-Hot Encoding:", rmse_one_hot)
print("RMSE with Frequency Encoding:", rmse_freq)

RMSE with One-Hot Encoding: 0.5663342573821032
RMSE with Frequency Encoding: 4.34529999116271


In [10]:
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'target': test_predictions_one_hot
})


submission_df.to_csv('/content/drive/MyDrive/linear_regression_prediction.csv', index=False)

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(train_features_one_hot, train_targets)
train_predictions_rf = model_rf.predict(train_features_one_hot)
test_predictions_rf = model_rf.predict(test_features_one_hot)

rmse_rf = np.sqrt(mean_squared_error(train_targets, train_predictions_rf))
print("Training RMSE with RandomForest:", rmse_rf)
submission_df_rf = pd.DataFrame({
    'id': test_df['id'],
    'target': test_predictions_rf
})
submission_df_rf.to_csv('/content/drive/MyDrive/withoutPCA_prediction_rf.csv', index=False)
print("RandomForest CSV file has been created and saved.")


KeyboardInterrupt: 

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA


pca = PCA(n_components=200)
train_features_pca = pca.fit_transform(train_features_one_hot)
test_features_pca = pca.transform(test_features_one_hot)

model_rf = RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42, n_jobs=-1)
model_rf.fit(train_features_pca, train_targets)

train_predictions_rf = model_rf.predict(train_features_pca)
rmse_rf = np.sqrt(mean_squared_error(train_targets, train_predictions_rf))
print("Training RMSE with RandomForest:", rmse_rf)

test_predictions_rf = model_rf.predict(test_features_pca)
submission_df_rf = pd.DataFrame({
    'id': test_df['id'],
    'target': test_predictions_rf
})
submission_df_rf.to_csv('/content/drive/MyDrive/estimators50_PCAprediction_rf.csv', index=False)
print("RandomForest CSV file has been created and saved.")

Training RMSE with RandomForest: 3.9445961902588302
RandomForest CSV file has been created and saved.


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA


pca = PCA(n_components=200)
train_features_pca = pca.fit_transform(train_features_one_hot)
test_features_pca = pca.transform(test_features_one_hot)

model_rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
model_rf.fit(train_features_pca, train_targets)

train_predictions_rf = model_rf.predict(train_features_pca)
rmse_rf = np.sqrt(mean_squared_error(train_targets, train_predictions_rf))
print("Training RMSE with RandomForest:", rmse_rf)

test_predictions_rf = model_rf.predict(test_features_pca)
submission_df_rf = pd.DataFrame({
    'id': test_df['id'],
    'target': test_predictions_rf
})
submission_df_rf.to_csv('/content/drive/MyDrive/estimators100_PCAprediction_rf.csv', index=False)
print("RandomForest CSV file has been created and saved.")

Training RMSE with RandomForest: 2.7864261661655534
RandomForest CSV file has been created and saved.


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

model_gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

model_gb.fit(train_features_one_hot, train_targets)

train_predictions_gb = model_gb.predict(train_features_one_hot)

test_predictions_gb = model_gb.predict(test_features_one_hot)

rmse_gb = np.sqrt(mean_squared_error(train_targets, train_predictions_gb))
print("Training RMSE with Gradient Boosting:", rmse_gb)

submission_df_gb = pd.DataFrame({
    'id': test_df['id'],
    'target': test_predictions_gb
})

submission_df_gb.to_csv('/content/drive/MyDrive/prediction_gb.csv', index=False)
print("Gradient Boosting CSV file has been created and saved to '/content/drive/MyDrive/prediction_gb.csv'")


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features_one_hot)
test_features_scaled = scaler.transform(test_features_one_hot)

model = Sequential([
    Dense(128, activation='relu', input_shape=(train_features_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  #no activation function
])

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(train_features_scaled, train_targets, epochs=50, batch_size=32, validation_split=0.2)

train_predictions_nn = model.predict(train_features_scaled)
rmse_nn = np.sqrt(mean_squared_error(train_targets, train_predictions_nn))
print("Training RMSE with Neural Network:", rmse_nn)

test_predictions_nn = model.predict(test_features_scaled)

submission_df_nn = pd.DataFrame({
    'id': test_df['id'],
    'target': test_predictions_nn.flatten()  #convert 2D array to 1D
})
submission_df_nn.to_csv('/content/drive/MyDrive/50epoch_prediction_nn.csv', index=False)
print("Neural Network CSV file has been created and saved.")


Epoch 1/50
204/204 [==============================] - 8s 25ms/step - loss: 3087.8518 - val_loss: 15102.3145
Epoch 2/50
204/204 [==============================] - 2s 11ms/step - loss: 4506.5898 - val_loss: 9698.2188
Epoch 3/50
204/204 [==============================] - 2s 11ms/step - loss: 9883.3604 - val_loss: 126625.7188
Epoch 4/50
204/204 [==============================] - 2s 11ms/step - loss: 19199.7676 - val_loss: 152216.1406
Epoch 5/50
204/204 [==============================] - 2s 11ms/step - loss: 6602.1489 - val_loss: 6555.6670
Epoch 6/50
204/204 [==============================] - 2s 11ms/step - loss: 39114.5703 - val_loss: 98433.6016
Epoch 7/50
204/204 [==============================] - 2s 11ms/step - loss: 3049.8008 - val_loss: 93177.6875
Epoch 8/50
204/204 [==============================] - 2s 11ms/step - loss: 6938.5073 - val_loss: 68105.7969
Epoch 9/50
204/204 [==============================] - 2s 11ms/step - loss: 437.9116 - val_loss: 75896.7344
Epoch 10/50
204/204 [======

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features_one_hot)
test_features_scaled = scaler.transform(test_features_one_hot)


model = Sequential([
    Dense(128, activation='relu', input_shape=(train_features_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(train_features_scaled, train_targets, epochs=300, batch_size=32, validation_split=0.2)
train_predictions_nn = model.predict(train_features_scaled)
rmse_nn = np.sqrt(mean_squared_error(train_targets, train_predictions_nn))
print("Training RMSE with Neural Network:", rmse_nn)

test_predictions_nn = model.predict(test_features_scaled)

submission_df_nn = pd.DataFrame({
    'id': test_df['id'],
    'target': test_predictions_nn.flatten()
})
submission_df_nn.to_csv('/content/drive/MyDrive/300epoch_prediction_nn.csv', index=False)
print("Neural Network CSV file has been created and saved.")

Epoch 1/300
204/204 [==============================] - 6s 23ms/step - loss: 3304.5686 - val_loss: 739.4025
Epoch 2/300
204/204 [==============================] - 2s 11ms/step - loss: 1546.9404 - val_loss: 234.9787
Epoch 3/300
204/204 [==============================] - 2s 11ms/step - loss: 2656.3718 - val_loss: 450.9965
Epoch 4/300
204/204 [==============================] - 2s 11ms/step - loss: 4161.5210 - val_loss: 64.4959
Epoch 5/300
204/204 [==============================] - 2s 11ms/step - loss: 2381.0791 - val_loss: 78.7011
Epoch 6/300
204/204 [==============================] - 2s 11ms/step - loss: 445.0100 - val_loss: 96.6510
Epoch 7/300
204/204 [==============================] - 2s 11ms/step - loss: 266.8411 - val_loss: 537.8660
Epoch 8/300
204/204 [==============================] - 2s 11ms/step - loss: 221.7504 - val_loss: 212.3617
Epoch 9/300
204/204 [==============================] - 2s 11ms/step - loss: 34.2403 - val_loss: 65.8672
Epoch 10/300
204/204 [========================

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features_one_hot)
test_features_scaled = scaler.transform(test_features_one_hot)

model = Sequential([
    Dense(128, activation='relu', input_shape=(train_features_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])


model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(train_features_scaled, train_targets, epochs=125, batch_size=32, validation_split=0.2)
train_predictions_nn = model.predict(train_features_scaled)
rmse_nn = np.sqrt(mean_squared_error(train_targets, train_predictions_nn))
print("Training RMSE with Neural Network:", rmse_nn)
test_predictions_nn = model.predict(test_features_scaled)

submission_df_nn = pd.DataFrame({
    'id': test_df['id'],
    'target': test_predictions_nn.flatten()
})
submission_df_nn.to_csv('/content/drive/MyDrive/125_epoch_prediction_nn.csv', index=False)
print("Neural Network CSV file has been created and saved.")

Epoch 1/125
204/204 [==============================] - 6s 22ms/step - loss: 2524.7468 - val_loss: 888.3648
Epoch 2/125
204/204 [==============================] - 2s 11ms/step - loss: 6005.3647 - val_loss: 4683.4810
Epoch 3/125
204/204 [==============================] - 2s 11ms/step - loss: 1624.9120 - val_loss: 4923.8491
Epoch 4/125
204/204 [==============================] - 2s 12ms/step - loss: 7844.4644 - val_loss: 17601.9102
Epoch 5/125
204/204 [==============================] - 2s 11ms/step - loss: 7287.2324 - val_loss: 21107.9395
Epoch 6/125
204/204 [==============================] - 2s 11ms/step - loss: 11529.3701 - val_loss: 696.6773
Epoch 7/125
204/204 [==============================] - 2s 11ms/step - loss: 1406.9497 - val_loss: 2487.6296
Epoch 8/125
204/204 [==============================] - 2s 11ms/step - loss: 1094.0067 - val_loss: 2183.1260
Epoch 9/125
204/204 [==============================] - 2s 11ms/step - loss: 133.2377 - val_loss: 1615.4587
Epoch 10/125
204/204 [======